In [1]:
# What verion of python
from platform import python_version
print(python_version())

3.9.13


In [2]:
# Anaconda details
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/usr/local/anaconda3/bin/python
3.9.13 (main, Aug 25 2022, 18:29:29) 
[Clang 12.0.0 ]
sys.version_info(major=3, minor=9, micro=13, releaselevel='final', serial=0)
